In [20]:
%%capture
!pip install deepxde

import torch
import deepxde as dde
import numpy as np

In [21]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_path = "/content/drive/MyDrive/MedRob"
utils_path = drive_path + "/ProjectMedicalRobotics/src"

In [22]:
# Problem Parameters:
rho = 1050 # tissue density [kg/(m^3)]
rho_b = 1043 # blood density [kg/(m^3)]
c = 3639 # specific heat of the tissue [J/(kg*K)]
c_b = 3825 # specific heat of blood [J/(kg*K)]
k_eff = 5 # termal conductivity of the tissue [W/(m*K)]
T_a = 37 # arterial blood temperature [C]
T_max = 45
Q = 0 # no internal source of heat
w_b = 2.22e-3 # blood perfusion rate [s^-1]
w_min = 0.43e-4 # perfusion rate min value [s^-1]
w_max = 3.8e-3 # perfusion rate max value [s^-1]
q_0 = 16
beta = 15
tf = 1800 # max time value
L_0 = 0.05 # length of interval -> 5 cm


alpha = rho*c/k_eff
a1 = 1800/(alpha*L_0*L_0)
a2 = (c_b*L_0*L_0)/k_eff
W = rho_b*w_b
print(f"Coefficient a1: {a1}")
print(f"Coefficient a2: {a2}")
print(f"Coefficient W: {W}")

Coefficient a1: 0.9421740666588152
Coefficient a2: 1.9125
Coefficient W: 2.3154600000000003


## Bio-Heat Equation:

### $ \rho c \ \partial_t T = k_{eff} \ \partial_{xx} T \ - \ \rho_b w_b c_b (T - Ta) \ + \ \frac{ \mathscr{Q}}{\rho c} $

## After Transformations:

### $ \partial_t \theta = \frac{k_{eff}}{\rho c L_0^2}\partial_{XX} \theta \ - \ \frac{\rho_b w_b c_b}{\rho c} \theta \ + \ \frac{ \mathscr{Q}}{\rho c (T_M \ - \ T_a)} $

### $ \partial_t \theta = \frac{1}{\alpha L_0^2}\partial_{XX} \theta \ - \ \frac{\rho_b w_b c_b}{\rho c} \theta \ + \ \frac{ \mathscr{Q}}{\rho c (T_M \ - \ T_a)} $

### $ \partial_t \theta = a_1\partial_{XX} \theta \ - \ a_2W \theta \ + \ \frac{ \mathscr{Q}}{\rho c (T_M \ - \ T_a)} $

In [ ]:
# define geometry (spatial) domain:

x_domain = dde.geometry.Interval(0, 1)

# define temporal domain:

t_domain = dde.geometry.TimeDomain(0, tf)

# fusion of domains:

xt_domain = dde.geometry.GeometryXTime(x_domain, t_domain)

In [ ]:
# express Bio-Heat Equation after applying transformations
def bio_heat_pde(x, y):
    # x: x[:,0] represent x, x[:,1] represent t
    # y: theta
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    # Q = 0
    return dy_t - a1*dy_xx + a2*W*y

In [24]:
# initial condition:

ic = dde.icbc.IC(
    xt_domain,
    lambda x: x**4*(q_0/(T_max-T_a))/4 + x*(beta/(T_max-T_a))*(x-1)**2,
    lambda _,
    on_initial: on_initial,
)

In [ ]:
# boundary condition: